<a href="https://www.kaggle.com/code/pardisnorouzi/da-project-new?scriptVersionId=122487671" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import numpy as np
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy import stats
from sklearn.model_selection import GridSearchCV
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/mobile-health/mhealth_raw_data.csv")
df

In [ ]:
# count null values in each column
null_counts = df.isnull().sum()

# print the null counts
print(null_counts)

In [ ]:
df[df == 0].count().sum()

In [ ]:
df = df.drop(['subject'], axis=1)
df = df[df['Activity']>0]
df

In [ ]:
df.info()

In [ ]:
# Get the current memory usage of the DataFrame
print(f"Memory usage before downcasting: {df.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB")

# Downcast the DataFrame as you have done before
df_downcasted = df.copy()

# Loop through each column in the DataFrame
for col in df_downcasted.columns:
    # Downcast float64 columns to float16
    if df_downcasted[col].dtype == 'float64':
        df_downcasted[col] = df_downcasted[col].astype('float16')

# Get the memory usage of the DataFrame after downcasting
print(f"Memory usage after downcasting: {df_downcasted.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB")

In [ ]:
# Select the columns to plot
cols_to_plot = ["alx", "aly", "alz", "glx", "gly", "glz", "arx", "ary", "arz", "grx", "gry", "grz"]

# Create a boxplot showing the data for each feature
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.boxplot(data=df[cols_to_plot], palette='coolwarm')
ax.set_title('Boxplot of Dataset (Excluding Activity Column)')
ax.set_ylabel('Value')
ax.set_xlabel('Feature')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Define the list of activities
activities = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]

# Define the number of rows and columns for the subplots
num_rows = 4
num_cols = 3

# Create the subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(18, 18))
axs = axs.ravel()

# Iterate over each activity
for i, act in enumerate(activities):
    # Filter the dataset by the current activity
    df_temp = df.loc[df.Activity == int(act)].copy()
    df_temp.reset_index(inplace=True)

    # Convert the time to seconds and start with 0
    df_temp['time_sec'] = df_temp.index/50

    # Plot the accelerometer readings on the current subplot
    axs[i].plot(df_temp['time_sec'], df_temp['alx'], color='red', alpha=0.5, label='alx')
    axs[i].plot(df_temp['time_sec'], df_temp['aly'], color='green', alpha=0.5, label='aly')
    axs[i].plot(df_temp['time_sec'], df_temp['alz'], color='blue', alpha=0.5, label='alz')
    axs[i].set_ylim(-20, 20)
    axs[i].set_title('Activity ' + act + ' - Accelerometer Readings (Left Arm)')
    axs[i].legend()
    axs[i].grid()

plt.show()

In [ ]:
sns.set_style('whitegrid') 

sns.countplot(x='Activity', data=df_downcasted) 


plt.title('Number of samples by activity') 

plt.show() 

In [ ]:
window = 1                 # window dimention in seconds 
fs = 50                    # frequency in Hz
windows_size = window * fs # number of samples for each time window

In [ ]:
!pip install tsfel

In [ ]:
import tsfel
import time

In [ ]:
FEATURES = True
SAVE_FEATURES = True

In [ ]:
if FEATURES:
    # Python3 program to Convert a list to dictionary 
    def Convert(lst):
        res_dct = {lst[i]: i for i in range(0, len(lst))}
        return res_dct

    # Driver code
    res_dct = Convert(df['Activity'].unique())
    res_dct 

In [ ]:
if FEATURES:
    df.Activity = df['Activity'].map(res_dct)
    del res_dct
    display(df['Activity'].unique())
    display(df.head())

In [ ]:
if FEATURES:
    t1 = time.time()
    # Retrieves a pre-defined feature configuration file to extract all available features
    # Available domains: "statistical"; "spectral"; "temporal"
    cfg_file = tsfel.get_features_by_domain()

    # Extract the Activity classes for each created window 
    tmp_class = df_downcasted['Activity'].values
    tmp_class = tmp_class[0:int(len(tmp_class)/windows_size)*windows_size].reshape((int(len(tmp_class)/windows_size),windows_size))
    tmp_class = pd.DataFrame(tmp_class).mode(axis=1)

    # Extract features
    tmp_features = tsfel.time_series_features_extractor(cfg_file, df_downcasted.iloc[0:,0:12], fs = fs, window_size=windows_size, header_names = df_downcasted.columns[0:12].values, n_jobs = -1)

    tmp_features['Activity'] = tmp_class
    df_features = tmp_features
    t2 = time.time()
    print('Elapsed time [s]:', np.round(t2-t1,4))

    del tmp_class, t1, t2, tmp_features

    display(df_features.head(5))
    display(df_features.shape)
    df_downcasted = df_features.copy()


In [ ]:
df_downcasted.isna().sum()

In [ ]:
df_downcasted.info()

In [ ]:
print(df_downcasted.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
corr_matrix = df_downcasted.corr()

# Create a larger figure
plt.figure(figsize=(14,12))

# Create a heatmap of the correlation matrix
sns.heatmap(corr_matrix, cmap='coolwarm')

# Show the plot
plt.show()

In [ ]:
to_drop = set()
corr_matrix = df_features.corr().abs()
upper = np.triu(corr_matrix, k=1)
upper_df = pd.DataFrame(upper, columns=corr_matrix.columns, index=corr_matrix.index)
for col in upper_df.columns:
    if any(upper_df[col] > 0.95):
        to_drop.add(col)

df_downcasted = df_downcasted.drop(columns=to_drop)



In [ ]:
df_downcasted.columns

In [ ]:
print(df_downcasted)
df_downcasted.info()


In [ ]:
X = df_downcasted.drop('Activity', axis=1)
y = df_downcasted['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Train set shape before scaling:", X_train.shape)
print("Test set shape before scaling:", X_test.shape)

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the fitted scaler
X_test_scaled = scaler.transform(X_test)

print("Train set shape after scaling:", X_train_scaled.shape)
print("Test set shape after scaling:", X_test_scaled.shape)



In [ ]:
import numpy as np

# assuming y is your target variable after scaling
unique, counts = np.unique(y, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Class counts:", class_counts)


In [ ]:
# Define the parameter grid for the Random Forest Classifier
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Create a Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Create a GridSearchCV object and fit it to the training data
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Get the best estimator from grid search and predict on test data
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_scaled)

In [ ]:
# Evaluate the model's performance
cm = confusion_matrix(y_test, y_pred)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
precision = cm[1, 1] / (cm[0, 1] + cm[1, 1])
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion matrix:\n", cm)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Precision:", precision)
print("F1 score:", f1)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df_downcasted.drop('Activity', axis=1)
y = df_downcasted['Activity']

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # change test_size to 0.2

# Define the parameter grid for the Random Forest Classifier
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Create a Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Create a GridSearchCV object and fit it to the training data
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Get the best estimator from grid search and predict on test data
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_scaled)

In [ ]:
# Evaluate the model's performance
cm = confusion_matrix(y_test, y_pred)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
precision = cm[1, 1] / (cm[0, 1] + cm[1, 1])
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion matrix:\n", cm)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Precision:", precision)
print("F1 score:", f1)
print("Accuracy:", accuracy)

In [ ]:
#SVM
# Define the parameter grid for the SVM
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

# Create an SVM classifier
svm = SVC(random_state=42)

# Create a GridSearchCV object and fit it to the training data
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Get the best estimator from grid search and predict on test data
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_scaled)

In [ ]:
# Evaluate the model's performance
cm = confusion_matrix(y_test, y_pred)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
precision = cm[1, 1] / (cm[0, 1] + cm[1, 1])
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion matrix:\n", cm)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Precision:", precision)
print("F1 score:", f1)
print("Accuracy:", accuracy)

In [ ]:
#Logistic Regression
# Define the parameter grid for the Logistic Regression Classifier
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Create a Logistic Regression Classifier
lr = LogisticRegression(random_state=42)

# Create a GridSearchCV object and fit it to the training data
grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Get the best estimator from grid search and predict on test data
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_scaled)

In [ ]:
# Evaluate the model's performance
cm = confusion_matrix(y_test, y_pred)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
precision = cm[1, 1] / (cm[0, 1] + cm[1, 1])
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion matrix:\n", cm)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Precision:", precision)
print("F1 score:", f1)
print("Accuracy:", accuracy)

In [ ]:
#Decision Tree
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define the parameter grid for the Decision Tree Classifier
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Create a Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state=42)

# Create a GridSearchCV object and fit it to the training data
grid_search = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Get the best estimator from grid search and predict on test data
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_scaled)

In [ ]:
# Evaluate the model's performance
cm = confusion_matrix(y_test, y_pred)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
precision = cm[1, 1] / (cm[0, 1] + cm[1, 1])
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion matrix:\n", cm)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Precision:", precision)
print("F1 score:", f1)
print("Accuracy:", accuracy)